#Predicting the probability of a song beeing chosen by Obama in his playlists

##Data exploration

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from google.colab import auth

# Will collect your credentials
auth.authenticate_user()

In [ ]:
# Query Bigquery
query = "SELECT * FROM `music-recommendation-system-24.ml_tables_eu.spotify_songs_obama_concat_csv_V1`"
project = "music-recommendation-system-24"

df_full = pd.read_gbq(query=query, project_id=project)

#An alternate table to use in order to retrieve immeasurably better results in the final playlist would be this one: music-recommendation-system-24.ml_tables_eu.spotify_songs_obama_concat_filtered_csv

<ipython-input-197-fac9842d289e>:5: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_full = pd.read_gbq(query=query, project_id=project)


In [ ]:
df_full.head()

,acousticness,danceability,duration_min,energy,genre,instrumentalness,key,liveness,loudness,mode,speechiness,track_album_name,track_album_release_year,track_artist,track_id,track_name,track_popularity,tempo,valence,in_obama_playlist
0,53.50,75.2,3.29,50.5,edm,0.0003,A,10.40,-7.621,major,25.30,I Like Me Better,2017,Lauv,1wjzFQodRWrPcQ0AnYnvQ9,I Like Me Better,83,92,41.9,0
1,10.60,44.8,3.11,78.4,edm,0.0108,A,66.80,-3.686,major,6.27,Night Visions,2012,Imagine Dragons,62yJjFtgkhUrXktIoSjgP2,Radioactive,78,136,23.6,0
2,1.62,62.3,4.07,73.4,edm,0.0002,A,14.50,-5.950,major,10.70,Dangerous Woman,2016,Ariana Grande,2meEiZKWkiN28gITzFwQo5,Into You,79,108,37.2,0
3,1.05,66.2,3.61,81.0,edm,7.3300,A,16.60,-9.396,major,3.16,Thanks A Million,2017,Jay Hardway,2gfToY9trgUdYIgPkc8zwR,Thanks A Million,0,120,45.5,0
4,3.33,53.0,3.15,66.1,edm,0.0286,A,7.32,-7.402,major,4.60,Guardian Angel,2017,Joe Ghost,6W8TXRdNeC8QkiOrR3oW4J,Guardian Angel,0,160,19.2,0


In [ ]:
df_full.shape

(28582, 20)

In [ ]:
df_full.isna().sum()

,0
acousticness,0
danceability,0
duration_min,0
energy,0
genre,0
instrumentalness,0
key,0
liveness,0
loudness,0
mode,0


In [ ]:
df_full["in_obama_playlist"].value_counts()

,count
in_obama_playlist,
0,28324
1,258


##Cleaning for ML

In [ ]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28582 entries, 0 to 28581
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   acousticness              28582 non-null  float64
 1   danceability              28582 non-null  float64
 2   duration_min              28582 non-null  float64
 3   energy                    28582 non-null  float64
 4   genre                     28582 non-null  object 
 5   instrumentalness          28582 non-null  float64
 6   key                       28582 non-null  object 
 7   liveness                  28582 non-null  float64
 8   loudness                  28582 non-null  float64
 9   mode                      28582 non-null  object 
 10  speechiness               28582 non-null  float64
 11  track_album_name          28582 non-null  object 
 12  track_album_release_year  28582 non-null  Int64  
 13  track_artist              28582 non-null  object 
 14  track_

In [ ]:
df = df_full.drop(columns=["track_name", "track_album_name"])

In [ ]:
df = df.set_index("track_id")

##Modeling

In [ ]:
from re import X
from sklearn.model_selection import train_test_split

# every columns in X variable except re_purchase which is our target
X = df.drop(columns=["in_obama_playlist"])
y = df["in_obama_playlist"]

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# store customers_ids for after
test_customers_ids = X_test.index

###Dealing with categorical features

In [ ]:
df.head(2)

,acousticness,danceability,duration_min,energy,genre,instrumentalness,key,liveness,loudness,mode,speechiness,track_album_release_year,track_artist,track_popularity,tempo,valence,in_obama_playlist
track_id,,,,,,,,,,,,,,,,,
1wjzFQodRWrPcQ0AnYnvQ9,53.5,75.2,3.29,50.5,edm,0.0003,A,10.4,-7.621,major,25.30,2017,Lauv,83,92,41.9,0
62yJjFtgkhUrXktIoSjgP2,10.6,44.8,3.11,78.4,edm,0.0108,A,66.8,-3.686,major,6.27,2012,Imagine Dragons,78,136,23.6,0


In [ ]:
X_train_cat = X_train[["track_artist", "genre", "key", "mode"]]
X_test_cat = X_test[["track_artist", "genre", "key", "mode"]]

In [ ]:
ohe = OneHotEncoder(drop='if_binary', sparse_output=False, handle_unknown="ignore")

In [ ]:
X_train_cat_encoded = ohe.fit_transform(X_train_cat)
X_test_cat_encoded = ohe.transform(X_test_cat)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded, columns=ohe.get_feature_names_out())
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded, columns=ohe.get_feature_names_out())

In [ ]:
X_train_cat_encoded_df.head()

,track_artist_!!!,track_artist_!deladap,"track_artist_""Dear Evan Hansen"" August 2018 Broadway Cast",track_artist_$ANFI,track_artist_$IFRA,track_artist_$uicideBoy$,track_artist_(G)I-DLE,track_artist_(Sandy) Alex G,track_artist_(dc),track_artist_*NSYNC,...,key_C,key_C#,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,mode_minor
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
X_train_cat_encoded_df.shape

(22865, 9474)

###Dealing with numerical features

In [ ]:
df.head(2)

,acousticness,danceability,duration_min,energy,genre,instrumentalness,key,liveness,loudness,mode,speechiness,track_album_release_year,track_artist,track_popularity,tempo,valence,in_obama_playlist
track_id,,,,,,,,,,,,,,,,,
1wjzFQodRWrPcQ0AnYnvQ9,53.5,75.2,3.29,50.5,edm,0.0003,A,10.4,-7.621,major,25.30,2017,Lauv,83,92,41.9,0
62yJjFtgkhUrXktIoSjgP2,10.6,44.8,3.11,78.4,edm,0.0108,A,66.8,-3.686,major,6.27,2012,Imagine Dragons,78,136,23.6,0


In [ ]:
X_train_num = X_train[["track_popularity", "track_album_release_year", "duration_min", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]
X_test_num = X_test[["track_popularity", "track_album_release_year", "duration_min", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]

In [ ]:
std = StandardScaler()

In [ ]:
X_train_num.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22865 entries, 33uV0G85zoAlzHYTJyaGRl to 2WwXhtq2knsErc7ojO0Dgb
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_popularity          22865 non-null  Int64  
 1   track_album_release_year  22865 non-null  Int64  
 2   duration_min              22865 non-null  float64
 3   danceability              22865 non-null  float64
 4   energy                    22865 non-null  float64
 5   loudness                  22865 non-null  float64
 6   speechiness               22865 non-null  float64
 7   acousticness              22865 non-null  float64
 8   instrumentalness          22865 non-null  float64
 9   liveness                  22865 non-null  float64
 10  valence                   22865 non-null  float64
 11  tempo                     22865 non-null  Int64  
dtypes: Int64(3), float64(9)
memory usage: 2.8+ MB


In [ ]:
X_train_num_scaled = std.fit_transform(X_train_num)
X_test_num_scaled = std.transform(X_test_num)

In [ ]:
X_train_num_scaled_df = pd.DataFrame(X_train_num_scaled, columns=X_train_num.columns)
X_test_num_scaled_df = pd.DataFrame(X_test_num_scaled, columns=X_test_num.columns)

###Concatenating numerical and categorical features

In [ ]:
X_train_processed = pd.concat([X_train_cat_encoded_df, X_train_num_scaled_df], axis=1)
X_test_processed = pd.concat([X_test_cat_encoded_df, X_test_num_scaled_df], axis=1)

##Fitting our logistic regression model

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(X_train_processed, y_train)

LogisticRegression()

In [ ]:
y_pred = logreg.predict(X_test_processed)

In [ ]:
logreg_accuracy = accuracy_score(y_test, y_pred)
logreg_accuracy

0.9940528249081686

In [ ]:
logreg.score(X_test_processed,y_test)

0.9940528249081686

##Fitting a random forrest model

In [ ]:
random_forest = RandomForestClassifier()

In [ ]:
random_forest.fit(X_train_processed, y_train)

RandomForestClassifier()

In [ ]:
random_forest_y_pred = random_forest.predict(X_test_processed)

In [ ]:
random_forest_accuracy = accuracy_score(y_test, random_forest_y_pred)
random_forest_accuracy
#Scores a bit lower than the logistical regression!

0.9937029910792373

##Fitting a Nearest Neighbors Model

In [ ]:
NearestNeighbors = KNeighborsClassifier()

In [ ]:
NearestNeighbors.fit(X_train_processed, y_train)

KNeighborsClassifier()

In [ ]:
NearestNeighbors_y_pred = NearestNeighbors.predict(X_test_processed)

In [ ]:
NearestNeighbors_accuracy = accuracy_score(y_test, NearestNeighbors_y_pred)
NearestNeighbors_accuracy
#Performs the lowest (but still exceptionally high!)

0.9917789050201155

##Predicting Obama's future musical taste with logistic regression

###Creating the song list

In [ ]:
# Query Bigquery
query = "SELECT * FROM `music-recommendation-system-24.ml_tables_eu.spotify_top_100_for_prediction_csv_V1`"
project = "music-recommendation-system-24"

df_full_prediction = pd.read_gbq(query=query, project_id=project)

<ipython-input-237-cfc479d20529>:5: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_full_prediction = pd.read_gbq(query=query, project_id=project)


In [ ]:
df_full_prediction.head(5)

,track_id,track_name,track_artist,track_popularity,track_album_name,track_album_release_year,duration_min,genre,record_label,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,3hRV0jL3vUpRrcy398teAU,The Night We Met,Lord Huron,88,Strange Trails,2015,3.47,folk,Play It Again Sam,44.8,...,A,-9.514,Major,4.52,96.80,27.20,63.8,9.98,174,3
1,0nrRP2bk19rLc0orkWPQk2,Wake Me Up,Avicii,83,TRUE,2013,4.12,edm,Universal Music AB,53.2,...,D,-5.697,Major,5.23,0.38,0.12,16.1,64.30,124,4
2,7BqHUALzNBTanL6OvsqmC1,Happier,"Marshmello,Bastille",82,Happier,2018,3.57,edm,Joytime Collective,68.7,...,F,-2.749,Major,4.52,19.10,0.00,16.7,67.10,100,4
3,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),"The Kid LAROI,Justin Bieber",78,STAY (with Justin Bieber),2021,2.36,pop,Columbia,59.1,...,C#,-5.484,Major,4.83,3.83,0.00,10.3,47.80,170,4
4,1BxfuPKGuaTgP7aM0Bbdwr,Cruel Summer,Taylor Swift,89,Lover,2019,2.97,pop,Taylor Swift,55.2,...,A,-5.707,Major,15.70,11.70,0.00,10.5,56.40,170,4


###Preparing the data for ML

In [ ]:
df_prediction = df_full_prediction.drop(columns=["track_name", "track_album_name"])

In [ ]:
df_prediction = df_prediction.set_index("track_id")

In [ ]:
df_prediction_cat = df_prediction[["track_artist", "genre", "key", "mode"]]
df_prediction_num = df_prediction[["track_popularity", "track_album_release_year", "duration_min", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]

In [ ]:
df_prediction_cat_encoded = ohe.transform(df_prediction_cat)
df_prediction_num_scaled = std.transform(df_prediction_num)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
df_prediction_cat_encoded_df = pd.DataFrame(df_prediction_cat_encoded, columns=ohe.get_feature_names_out())
df_prediction_num_scaled_df = pd.DataFrame(df_prediction_num_scaled, columns=df_prediction_num.columns)

In [ ]:
df_prediction_processed = pd.concat([df_prediction_cat_encoded_df, df_prediction_num_scaled_df], axis=1)

###Predicting

In [ ]:
logreg_predictions = logreg.predict(df_prediction_processed)
logreg_predictions

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [ ]:
proba = pd.DataFrame(logreg.predict_proba(df_prediction_processed), columns=["Obama dislikes", "Obama likes"])

In [ ]:
proba["Obama likes"]=round(proba["Obama likes"]*100,2)
proba["Obama dislikes"]=round(proba["Obama dislikes"]*100,2)

proba.head()

,Obama dislikes,Obama likes
0,29.04,70.96
1,99.55,0.45
2,99.41,0.59
3,99.44,0.56
4,99.10,0.90


In [ ]:
df_full_prediction_proba = df_full_prediction.merge(proba, left_index=True, right_index=True)

In [ ]:
df_full_prediction_proba = df_full_prediction_proba.sort_values(by="Obama likes", ascending=False)
df_full_prediction_proba = df_full_prediction_proba[["track_name", "track_artist", "Obama likes"]]
df_full_prediction_proba

,track_name,track_artist,Obama likes
0,The Night We Met,Lord Huron,70.96
33,Kill Bill,SZA,7.61
30,I Wanna Be Yours,Arctic Monkeys,4.11
34,No Role Modelz,J. Cole,3.94
23,Heat Waves,Glass Animals,3.28
7,Memories,Maroon 5,2.44
32,Without Me,Eminem,2.16
14,Too Good At Goodbyes,Sam Smith,2.11
36,INDUSTRY BABY (feat. Jack Harlow),"Lil Nas X,Jack Harlow",2.02
12,Cold Heart - PNAU Remix,"Elton John,Dua Lipa,PNAU",1.35
